In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import stone_pose_hori_learning_used
from scoop.msg import stone_pose_ver_learning_used
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
rospy.init_node('scoop', anonymous=True) #initialize the node



In [2]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1610965417.580309]: Client test: Starting sending goals


In [3]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

True

In [4]:
import serial
import signal
import time

serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass
def softtip():
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1.5)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2090')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)

    
def hardtip():
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2180')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1.5)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2090')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)

In [5]:
def go_to_home():
    
    Hong_joint0 = radians(101.23)
    Hong_joint1 = radians(-83.77)
    Hong_joint2 = radians(-130.28)
    Hong_joint3 = radians(-55.97)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.73)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.70)
    Hong_joint1 = radians(-75.56)
    Hong_joint2 = radians(-116.96)
    Hong_joint3 = radians(-77.49)
    Hong_joint4 = radians(89.42)
    Hong_joint5 = radians(-78.13)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [6]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    if pose_z==0.0:
        return [None, None, None]
    rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
    camPstone = np.array([pose_x, pose_y, pose_z, 1])
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
    tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
    worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
    worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
    return [worldTstone[0]+0.01953-0.0032, worldTstone[1]+0.02148-0.0035+0.006, worldTstone[2]]

In [7]:
import json
def hori_stone_pose_callback(msg):
    global hori_pose_Go_stone_set, hori_Go_stone_number, yaw_pitch_normal_set, hori_box_set, hori_json_error
    hori_Go_stone_number = int(msg.hori_Go_stone_number)
    if hori_Go_stone_number == 0:
        hori_pose_Go_stone_set = []
        Go_stone_dir_sorted = []
        yaw_pitch_normal_set = []
        hori_box_set = []
    else:
        #try:
        if True:
            pose_x = msg.x
            pose_y = msg.y
            pose_z = msg.z
            yaw = msg.yaw
            pitch = msg.pitch
            normal = msg.normal
            hori_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
            yaw_pitch_normal_set.append([yaw,pitch,normal])
            box = json.loads(msg.box)
            box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
            hori_box_set.append(box)
        #except:
            #hori_json_error = True
hori_pose_topic = '/hori_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(hori_pose_topic, stone_pose_hori_learning_used, hori_stone_pose_callback)

In [8]:
import json
def ver_stone_pose_callback(msg):
    global ver_pose_Go_stone_set, ver_Go_stone_number, ver_gripper_direction_set, ver_gripper_aperture_set, ver_json_error
    ver_Go_stone_number = int(msg.ver_Go_stone_number)
    if ver_Go_stone_number == 0:
        ver_pose_Go_stone_set = []
        ver_gripper_direction_set = []
        ver_gripper_aperture_set = []
    else:
        try:
            pose_x = msg.x
            pose_y = msg.y
            pose_z = msg.z 
            box = json.loads(msg.box)
            ver_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
            box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
            #print box
            length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
            length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
            if length1 > length2:
                try:
                    if abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2)<abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2):
                        temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2
                    else:
                        temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2
                    ver_gripper_direction_set.append(temp_angle)
                except:
                    ver_gripper_direction_set.append(0)
                gripper_aperture_set.append(length2+0.006)
            else:
                try:
                    if abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2)<abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2):
                        temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2
                    else:
                        temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2
                    ver_gripper_direction_set.append(temp_angle)
                except:
                    ver_gripper_direction_set.append(0)
                ver_gripper_aperture_set.append(length1+0.006)       
        except:
            ver_json_error = True
ver_pose_topic = '/ver_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(ver_pose_topic, stone_pose_ver_learning_used, ver_stone_pose_callback)

In [9]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count, cv2_img, cv2_depth_img
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/terry/Mask_RCNN/samples/domino/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/terry/Mask_RCNN/samples/domino/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/terry/Mask_RCNN/samples/domino/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [10]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [11]:
def poke_grasp_execute(Go_index):
    print 'Go_index', Go_index
    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=0.033, speed=0.5, force=10, block=True)
    ####################### change tcp pose ############################
    rob.set_tcp((0.006, 0, 0.3734, 0, 0, 0))
    rospy.sleep(0.5)
    ####################################################################
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.3734,-0.006,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    print tcpTee

    n_normalize = normal / np.linalg.norm(normal)
    print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    print "tcpTnormal", tcpTnormal
    worldTnormal = [tcpTnormal[1][0,0], tcpTnormal[0][0,0], tcpTnormal[-2][0,0]]

    box = hori_box_set[Go_index]
    length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
    length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
    if length1 > length2:
        midpoint0 = [(box[1][0]+box[2][0])/2, (box[1][1]+box[2][1])/2]
        midpoint1 = [(box[0][0]+box[3][0])/2, (box[0][1]+box[3][1])/2]
    else:
        midpoint0 = [(box[0][0]+box[1][0])/2, (box[0][1]+box[1][1])/2]
        midpoint1 = [(box[2][0]+box[3][0])/2, (box[2][1]+box[3][1])/2]
    center = [(midpoint0[0]+midpoint1[0])/2, (midpoint0[1]+midpoint1[1])/2]
    dir0 = [midpoint0[0]-center[0], midpoint0[1]-center[1], 0]
    dir1 = [midpoint1[0]-center[0], midpoint1[1]-center[1], 0] 
    if np.dot(dir0, worldTnormal)>0:
        high_to_low_dir = dir0
    else:
        high_to_low_dir = dir1
    rotate_angle = atan2(high_to_low_dir[1], high_to_low_dir[0])-atan2(1,0)
    if rotate_angle > 0:
        rotate_angle = rotate_angle % (2*pi)
    else:
        rotate_angle = rotate_angle % (-2*pi)
    rotate_angle0 = -rotate_angle
    rotate_angle0_matrix = np.array([[cos(rotate_angle0), -sin(rotate_angle0), 0], [sin(rotate_angle0), cos(rotate_angle0), 0], [0,0,1]])
    tcp2normal_after_rotation0 = np.matmul(np.linalg.inv(rotate_angle0_matrix), tcpTnormal[0:3,:])
    tcp2normal_after_rotation0 = [tcp2normal_after_rotation0[0][0,0], tcp2normal_after_rotation0[1][0,0], tcp2normal_after_rotation0[2][0,0]]
    rotate_angle1 = atan(-tcp2normal_after_rotation0[1]/tcp2normal_after_rotation0[2])
    rotate_angle2 = asin(-tcp2normal_after_rotation0[0])
    
    if str(rotate_angle0) == 'nan' or str(rotate_angle1) == 'nan' or str(rotate_angle2) == 'nan':
        return False
    hardtip()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1180')
    signal.setitimer(signal.ITIMER_REAL, 0)
    move = m3d.Transform((tcpTstone[0,0]+0.001, tcpTstone[1,0]-0.004, 0, 0, 0, 0))    
    rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
    rob.translate_tool((0, 0, tcpTstone[2,0]-0.015), acc=0.5, vel=0.8)
    #k=l+m
    rob.movel_tool((0,0,0,0,0,rotate_angle0), acc=0.5, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,rotate_angle1,0,0), acc=0.5, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,0,rotate_angle2,0), acc=0.5, vel=0.8, wait=True)
    ###########################Choose d and psi######################################
    rob.translate_tool((0.01, 0, 0), acc=0.06, vel=0.08)
    psi = radians(20)
    rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
    #################################################################################
    rob.translate_tool((0, 0, 0.04), acc=0.01, vel=0.05, wait=False)
    '''
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.force.z
    time0 = time.time()
    IsStack = False
    num_large_force = 0
    while True:
        if num_large_force == 5:
            IsStack = True
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.force.z<ini_wrench-5:   # need change
            num_large_force += 1
        if time.time()-time0>3.7:  #3.7
            IsStack = False
            break
    print 'IsStack', IsStack
    
    if IsStack==False:
        rob.translate_tool((0, 0, 0.005), acc=0.01, vel=0.05, wait=False)
        for k0 in range(0, -31, -2):
            signal.setitimer(signal.ITIMER_REAL, 0.001)
            ser.write('1'+str(100+k0))
            signal.setitimer(signal.ITIMER_REAL, 0)
            time.sleep(0.1)
        Robotiq.goto(robotiq_client, pos=0.006, speed=0.5, force=10, block=False)
        rospy.sleep(.5)
        rob.translate_tool((0, 0, -0.08), acc=0.3, vel=0.8)
        rob.set_tcp((0, 0, 0, 0, 0, 0))
        rospy.sleep(0.5)
        go_to_end_place()
        rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
        Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
        softtip()
        rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
    else:
        switching_stone_coordinate_set.append(hori_pose_Go_stone_set[Go_index])
        rob.translate_tool((0, 0, -0.08), acc=0.3, vel=0.8)
        rob.set_tcp((0, 0, 0, 0, 0, 0))
        rospy.sleep(0.5)        
        go_to_safe_place()
    '''
    return True

In [12]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time

def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

def pos_flength_adjust_left_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_l):
    theta=theta*pi/180
    current_flength_r = flength_l + (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    transforming_matrix_w_g = np.array([[sin(theta), -cos(theta), -current_left_fingertip_pos_g[0]*sin(theta)+current_left_fingertip_pos_g[1]*cos(theta)], [cos(theta), sin(theta), -current_left_fingertip_pos_g[0]*cos(theta)-current_left_fingertip_pos_g[1]*sin(theta)], [0, 0, 1]])
    current_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[current_right_fingertip_pos_g[0]], [current_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    next_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_left_fingertip_pos_g[0]], [next_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_right_fingertip_pos_g[0]], [next_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    translation_dir_dis = [-next_left_fingertip_pos_w[0], -next_left_fingertip_pos_w[1]]
    next_right_fingertip_pos_after_trans_w = [next_right_fingertip_pos_w[0]+translation_dir_dis[0], next_right_fingertip_pos_w[1]+translation_dir_dis[1]]
    right_finger_shorten_distance=abs(next_right_fingertip_pos_after_trans_w[1])/sin(theta)
    translation_dir_dis[0] = translation_dir_dis[0]+(2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/sin(theta)-(next_right_fingertip_pos_after_trans_w[0]-right_finger_shorten_distance*cos(theta))
    return translation_dir_dis, right_finger_shorten_distance

'''
def scoop1(Go_index):
    print 'Go_index', Go_index
    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=9*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
    ####################### change tcp pose ############################
    rob.set_tcp((0.013, 0, 0.3534, 0, 0, 0))
    rospy.sleep(0.5)
    ####################################################################
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.3534,-0.013,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    print tcpTee
    n_normalize = normal / np.linalg.norm(normal)
    print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    print "tcpTnormal", tcpTnormal
    worldTnormal = [tcpTnormal[1][0,0], tcpTnormal[0][0,0], tcpTnormal[-2][0,0]]

    box = hori_box_set[Go_index]
    length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
    length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
    if length1 > length2:
        midpoint0 = [(box[1][0]+box[2][0])/2, (box[1][1]+box[2][1])/2]
        midpoint1 = [(box[0][0]+box[3][0])/2, (box[0][1]+box[3][1])/2]
    else:
        midpoint0 = [(box[0][0]+box[1][0])/2, (box[0][1]+box[1][1])/2]
        midpoint1 = [(box[2][0]+box[3][0])/2, (box[2][1]+box[3][1])/2]
    center = [(midpoint0[0]+midpoint1[0])/2, (midpoint0[1]+midpoint1[1])/2]
    dir0 = [midpoint0[0]-center[0], midpoint0[1]-center[1], 0]
    dir1 = [midpoint1[0]-center[0], midpoint1[1]-center[1], 0] 
    if np.dot(dir0, worldTnormal)>0:
        high_to_low_dir = dir0
    else:
        high_to_low_dir = dir1
    rotate_angle = atan2(high_to_low_dir[1], high_to_low_dir[0])-atan2(1,0)
    if rotate_angle > 0:
        rotate_angle = rotate_angle % (2*pi)
    else:
        rotate_angle = rotate_angle % (-2*pi)
    rotate_angle0 = -rotate_angle
    rotate_angle0_matrix = np.array([[cos(rotate_angle0), -sin(rotate_angle0), 0], [sin(rotate_angle0), cos(rotate_angle0), 0], [0,0,1]])
    tcp2normal_after_rotation0 = np.matmul(np.linalg.inv(rotate_angle0_matrix), tcpTnormal[0:3,:])
    tcp2normal_after_rotation0 = [tcp2normal_after_rotation0[0][0,0], tcp2normal_after_rotation0[1][0,0], tcp2normal_after_rotation0[2][0,0]]
    rotate_angle1 = atan(-tcp2normal_after_rotation0[1]/tcp2normal_after_rotation0[2])
    rotate_angle2 = asin(-tcp2normal_after_rotation0[0])
    
    if str(rotate_angle0) == 'nan' or str(rotate_angle1) == 'nan' or str(rotate_angle2) == 'nan':
        return False
    softtip()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1075')
    signal.setitimer(signal.ITIMER_REAL, 0)
    move = m3d.Transform((tcpTstone[0,0]+0.001, tcpTstone[1,0]-0.004, 0, 0, 0, 0))    
    rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
    rob.translate_tool((0, 0, tcpTstone[2,0]-0.015), acc=0.5, vel=0.8)
    #k=l+m
    rob.movel_tool((0,0,0,0,0,rotate_angle0), acc=0.5, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,rotate_angle1,0,0), acc=0.5, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,0,rotate_angle2,0), acc=0.5, vel=0.8, wait=True)
    ###########################Choose d and psi######################################
    rob.translate_tool((0.0165, 0, 0), acc=0.06, vel=0.08)
    psi = radians(30)
    rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
    #################################################################################
    rob.translate_tool((0, 0, 0.02), acc=0.01, vel=0.05, wait=False)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.force.z
    time0 = time.time()
    IsStack = False
    num_large_force = 0
    while True:
        if num_large_force == 3:
            IsStack = True
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.force.z<ini_wrench-3:   # need change
            num_large_force += 1
        if time.time()-time0>3.7:  #3.7
            IsStack = False
            break
    print 'IsStack', IsStack
    time.sleep(0.5)
    current_fingerlength_int=75   
    angle_step=0.2
    for angle in np.arange(9,-2,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        #if True:
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-2*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(0.5)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1180')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=0.013, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    ser.write('1010')
    rospy.sleep(1)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)     

    return True
'''

'\ndef scoop1(Go_index):\n    print \'Go_index\', Go_index\n    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set\n    pose_x = hori_pose_Go_stone_set[Go_index][0]\n    pose_y = hori_pose_Go_stone_set[Go_index][1]\n    pose_z = hori_pose_Go_stone_set[Go_index][2]\n    normal=yaw_pitch_normal_set[Go_index][2]\n    go_to_safe_place()\n    Robotiq.goto(robotiq_client, pos=9*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)\n    ####################### change tcp pose ############################\n    rob.set_tcp((0.013, 0, 0.3534, 0, 0, 0))\n    rospy.sleep(0.5)\n    ####################################################################\n    worldPstone = np.array([pose_x, pose_y, pose_z, 1])\n    print \'worldPstone\', worldPstone\n    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))\n    print "tcpTstone", tcpTstone\n    eeTcam = m3d.Transform()\n    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03

In [13]:
'''
def scoop2(Go_index):
    print 'Go_index', Go_index
    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
    ####################### change tcp pose ############################
    rob.set_tcp((0.002, 0, 0.3476, 0, 0, 0))
    rospy.sleep(0.5)
    ####################################################################
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.3476,-0.002,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    print tcpTee
    n_normalize = normal / np.linalg.norm(normal)
    print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    print "tcpTnormal", tcpTnormal
    worldTnormal = [tcpTnormal[1][0,0], tcpTnormal[0][0,0], tcpTnormal[-2][0,0]]

    box = hori_box_set[Go_index]
    length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
    length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
    if length1 < length2:
        midpoint0 = [(box[1][0]+box[2][0])/2, (box[1][1]+box[2][1])/2]
        midpoint1 = [(box[0][0]+box[3][0])/2, (box[0][1]+box[3][1])/2]
    else:
        midpoint0 = [(box[0][0]+box[1][0])/2, (box[0][1]+box[1][1])/2]
        midpoint1 = [(box[2][0]+box[3][0])/2, (box[2][1]+box[3][1])/2]
    center = [(midpoint0[0]+midpoint1[0])/2, (midpoint0[1]+midpoint1[1])/2]
    dir0 = [midpoint0[0]-center[0], midpoint0[1]-center[1], 0]
    dir1 = [midpoint1[0]-center[0], midpoint1[1]-center[1], 0] 
    if np.dot(dir0, worldTnormal)>0:
        high_to_low_dir = dir0
    else:
        high_to_low_dir = dir1
    rotate_angle = atan2(high_to_low_dir[1], high_to_low_dir[0])-atan2(1,0)
    if rotate_angle > 0:
        rotate_angle = rotate_angle % (2*pi)
    else:
        rotate_angle = rotate_angle % (-2*pi)
    rotate_angle0 = -rotate_angle
    rotate_angle0_matrix = np.array([[cos(rotate_angle0), -sin(rotate_angle0), 0], [sin(rotate_angle0), cos(rotate_angle0), 0], [0,0,1]])
    tcp2normal_after_rotation0 = np.matmul(np.linalg.inv(rotate_angle0_matrix), tcpTnormal[0:3,:])
    tcp2normal_after_rotation0 = [tcp2normal_after_rotation0[0][0,0], tcp2normal_after_rotation0[1][0,0], tcp2normal_after_rotation0[2][0,0]]
    rotate_angle1 = atan(-tcp2normal_after_rotation0[1]/tcp2normal_after_rotation0[2])
    rotate_angle2 = asin(-tcp2normal_after_rotation0[0])
    
    if str(rotate_angle0) == 'nan' or str(rotate_angle1) == 'nan' or str(rotate_angle2) == 'nan':
        return False
    softtip()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1033')
    signal.setitimer(signal.ITIMER_REAL, 0)
    move = m3d.Transform((tcpTstone[0,0]+0.001, tcpTstone[1,0]-0.004, 0, 0, 0, 0))    
    rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
    rob.translate_tool((0, 0, tcpTstone[2,0]-0.018), acc=0.5, vel=0.8)
    #k=l+m
    rob.movel_tool((0,0,0,0,0,rotate_angle0), acc=0.5, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,rotate_angle1,0,0), acc=0.5, vel=0.8, wait=True)
    rob.movel_tool((0,0,0,0,rotate_angle2,0), acc=0.5, vel=0.8, wait=True)
    ###########################Choose d and psi######################################
    rob.translate_tool((0.009, 0, 0), acc=0.06, vel=0.08)
    psi = radians(30)
    rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
    #################################################################################
    rob.translate_tool((0, 0, 0.02), acc=0.01, vel=0.05, wait=False)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.force.z
    time0 = time.time()
    IsStack = False
    num_large_force = 0
    while True:
        if num_large_force == 3:
            IsStack = True
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.force.z<ini_wrench-3:   # need change
            num_large_force += 1
        if time.time()-time0>3.7:  #3.7
            IsStack = False
            break
    print 'IsStack', IsStack
    time.sleep(0.5)
    current_fingerlength_int=33   
    angle_step=0.2
    for angle in np.arange(3,-2,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        #if True:
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-2*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(0.5)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1180')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=0.013, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    ser.write('1010')
    rospy.sleep(1)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)     

    return True
'''

'\ndef scoop2(Go_index):\n    print \'Go_index\', Go_index\n    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set\n    pose_x = hori_pose_Go_stone_set[Go_index][0]\n    pose_y = hori_pose_Go_stone_set[Go_index][1]\n    pose_z = hori_pose_Go_stone_set[Go_index][2]\n    normal=yaw_pitch_normal_set[Go_index][2]\n    go_to_safe_place()\n    Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)\n    ####################### change tcp pose ############################\n    rob.set_tcp((0.002, 0, 0.3476, 0, 0, 0))\n    rospy.sleep(0.5)\n    ####################################################################\n    worldPstone = np.array([pose_x, pose_y, pose_z, 1])\n    print \'worldPstone\', worldPstone\n    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))\n    print "tcpTstone", tcpTstone\n    eeTcam = m3d.Transform()\n    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03

In [14]:
'''
def scoop3(Go_index):
    print 'Go_index', Go_index
    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
    ####################### change tcp pose ############################
    rob.set_tcp((0.013, 0, 0.340, 0, 0, 0))
    rospy.sleep(0.5)
    ####################################################################
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.340,-0.013,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    print tcpTee
    n_normalize = normal / np.linalg.norm(normal)
    print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    print "tcpTnormal", tcpTnormal
    worldTnormal = [tcpTnormal[1][0,0], tcpTnormal[0][0,0], tcpTnormal[-2][0,0]]

    box = hori_box_set[Go_index]
    length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
    length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
    if length1 > length2:
        midpoint0 = [(box[1][0]+box[2][0])/2, (box[1][1]+box[2][1])/2]
        midpoint1 = [(box[0][0]+box[3][0])/2, (box[0][1]+box[3][1])/2]
    else:
        midpoint0 = [(box[0][0]+box[1][0])/2, (box[0][1]+box[1][1])/2]
        midpoint1 = [(box[2][0]+box[3][0])/2, (box[2][1]+box[3][1])/2]
    center = [(midpoint0[0]+midpoint1[0])/2, (midpoint0[1]+midpoint1[1])/2]
    dir0 = [midpoint0[0]-center[0], midpoint0[1]-center[1], 0]
    dir1 = [midpoint1[0]-center[0], midpoint1[1]-center[1], 0] 
    if np.dot(dir0, worldTnormal)>0:
        high_to_low_dir = dir0
    else:
        high_to_low_dir = dir1
    rotate_angle = atan2(high_to_low_dir[1], high_to_low_dir[0])-atan2(1,0)
    if rotate_angle > 0:
        rotate_angle = rotate_angle % (2*pi)
    else:
        rotate_angle = rotate_angle % (-2*pi)
    rotate_angle0 = -rotate_angle
    if str(rotate_angle0) == 'nan':
        return False
    softtip()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    move = m3d.Transform((tcpTstone[0,0]+0.001, tcpTstone[1,0]-0.004, 0, 0, 0, 0))    
    rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
    rob.movel_tool((0,0,0,0,0,rotate_angle0), acc=0.5, vel=0.8, wait=True)
    rob.translate_tool((0.0165, 0, 0), acc=0.06, vel=0.08)
    rob.set_tcp((-0.0267, 0, 0.340, 0, 0, 0))
    rospy.sleep(0.5)
    rob.movel_tool((0,0,0,0,pi/6,0), acc=0.5, vel=0.8, wait=True)
    rob.translate((0, 0, -(tcpTstone[2,0]-0.015)), acc=0.5, vel=0.8)
    rob.translate((0, 0, -0.02), acc=0.01, vel=0.05, wait=False)   
                
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    time0 = time.time()
    num_large_force = 0
    while True:
        if num_large_force == 3:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.1:
            num_large_force += 1
        if time.time()-time0>2.8: # need change
            break
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    for current_finger_length in np.arange(0,184,4):
        signal.setitimer(signal.ITIMER_REAL, 0.001)
        ser.write('1'+str(current_finger_length))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.2)        
        current_fingerlength_int=current_finger_length
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.3:
            break
    angle_step=0.2
    for angle in np.arange(10,-2,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        #if True:
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-2*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(0.5)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1180')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=0.013, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    ser.write('1010')
    rospy.sleep(1)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True) 

    return True
'''

'\ndef scoop3(Go_index):\n    print \'Go_index\', Go_index\n    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set\n    pose_x = hori_pose_Go_stone_set[Go_index][0]\n    pose_y = hori_pose_Go_stone_set[Go_index][1]\n    pose_z = hori_pose_Go_stone_set[Go_index][2]\n    normal=yaw_pitch_normal_set[Go_index][2]\n    go_to_safe_place()\n    Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)\n    ####################### change tcp pose ############################\n    rob.set_tcp((0.013, 0, 0.340, 0, 0, 0))\n    rospy.sleep(0.5)\n    ####################################################################\n    worldPstone = np.array([pose_x, pose_y, pose_z, 1])\n    print \'worldPstone\', worldPstone\n    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))\n    print "tcpTstone", tcpTstone\n    eeTcam = m3d.Transform()\n    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03,

In [25]:
def scoop4(Go_index):
    print 'Go_index', Go_index
    global hori_pose_Go_stone_set, yaw_pitch_normal_set, switching_stone_coordinate_set, hori_box_set
    pose_x = hori_pose_Go_stone_set[Go_index][0]
    pose_y = hori_pose_Go_stone_set[Go_index][1]
    pose_z = hori_pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=6*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
    ####################### change tcp pose ############################
    rob.set_tcp((0.002, 0, 0.340, 0, 0, 0))
    rospy.sleep(0.5)
    ####################################################################
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.340,-0.002,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    print tcpTee
    n_normalize = normal / np.linalg.norm(normal)
    print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    print "tcpTnormal", tcpTnormal
    worldTnormal = [tcpTnormal[1][0,0], tcpTnormal[0][0,0], tcpTnormal[-2][0,0]]

    box = hori_box_set[Go_index]
    length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
    length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
    if length1 < length2:
        midpoint0 = [(box[1][0]+box[2][0])/2, (box[1][1]+box[2][1])/2]
        midpoint1 = [(box[0][0]+box[3][0])/2, (box[0][1]+box[3][1])/2]
    else:
        midpoint0 = [(box[0][0]+box[1][0])/2, (box[0][1]+box[1][1])/2]
        midpoint1 = [(box[2][0]+box[3][0])/2, (box[2][1]+box[3][1])/2]
    center = [(midpoint0[0]+midpoint1[0])/2, (midpoint0[1]+midpoint1[1])/2]
    dir0 = [midpoint0[0]-center[0], midpoint0[1]-center[1], 0]
    dir1 = [midpoint1[0]-center[0], midpoint1[1]-center[1], 0] 
    if np.dot(dir0, worldTnormal)>0:
        high_to_low_dir = dir0
    else:
        high_to_low_dir = dir1
    rotate_angle = atan2(high_to_low_dir[1], high_to_low_dir[0])-atan2(1,0)
    if rotate_angle > 0:
        rotate_angle = rotate_angle % (2*pi)
    else:
        rotate_angle = rotate_angle % (-2*pi)
    rotate_angle0 = -rotate_angle
    if str(rotate_angle0) == 'nan':
        return False
    softtip()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    move = m3d.Transform((tcpTstone[0,0]+0.001, tcpTstone[1,0]-0.004, 0, 0, 0, 0))    
    rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
    rob.movel_tool((0,0,0,0,0,rotate_angle0), acc=0.5, vel=0.8, wait=True)
    rob.translate_tool((0.008, 0, 0), acc=0.06, vel=0.08)
    rob.set_tcp((-0.018, 0, 0.340, 0, 0, 0))
    rospy.sleep(0.5)
    rob.movel_tool((0,0,0,0,pi/6,0), acc=0.5, vel=0.8, wait=True)
    rob.translate((0, 0, -(tcpTstone[2,0]-0.015)), acc=0.5, vel=0.8)
    rob.translate((0, 0, -0.02), acc=0.01, vel=0.05, wait=False)   
                
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    time0 = time.time()
    num_large_force = 0
    while True:
        if num_large_force == 3:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.1:
            num_large_force += 1
        if time.time()-time0>2.8: # need change
            break
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    for current_finger_length in np.arange(0,40,4):
        signal.setitimer(signal.ITIMER_REAL, 0.001)
        ser.write('1'+str(current_finger_length))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.2)        
        current_fingerlength_int=current_finger_length
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.3:
            break
    angle_step=0.2
    for angle in np.arange(6,-2,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        #if True:
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-2*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(0.5)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1180')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=0.013, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    ser.write('1010')
    rospy.sleep(1)
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True) 

    return True

In [26]:
def find_alpha_from_distance(distance, l0, l1, l2l, l2r):
    alpha = asin((distance+l2l+l2r-2*l0)/2/l1)*180/pi
    return alpha

def direct_pick_execute(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    go_to_safe_place()
    hardtip()
    Robotiq.goto(robotiq_client, pos=0.75*gripper_aperture_set[Go_index]+0.0105, speed=0.1, force=10, block=True)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    pos_x = rob.get_pose().pos.x
    pos_y = rob.get_pose().pos.y
    pos_z = rob.get_pose().pos.z
    rospy.sleep(0.5)
    rob.translate((ver_pose_Go_stone_set[Go_index][0]-pos_x-0.005,ver_pose_Go_stone_set[Go_index][1]-pos_y+0.0015,0), acc=0.8, vel=2, wait=True)
    rob.movel_tool((0,0,0,0,0,gripper_direction_set[Go_index]),acc=0.8, vel=2)
    rob.translate_tool((0.005,0,0),acc=0.1, vel=0.2)
    #rob.translate((0,0,-0.025), acc=0.1, vel=0.4, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.7,-0.035)), acc=0.1, vel=0.1, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.6,-0.035)), acc=0.001, vel=0.01, wait=True)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.1, force=0.3, block=False)
    rospy.sleep(1)
    rob.translate((0, 0, 0.08), acc=0.3, vel=0.8)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.019, speed=0.5, force=10, block=True)
    softtip()
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)

In [27]:
from shapely.affinity import rotate
from shapely.affinity import translate
from shapely.geometry import Point
from shapely.geometry import LinearRing

def direct_pick_collision_check(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    finger_profile = [[0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, -gripper_aperture_set[Go_index]/2-0.007], [0.01, -gripper_aperture_set[Go_index]/2-0.007]]
    finger_profile = rotate(LinearRing(finger_profile), -gripper_direction_set[Go_index], origin=Point([0,0]), use_radians=True)
    finger_profile = translate(LinearRing(finger_profile), xoff=ver_pose_Go_stone_set[Go_index][0], yoff=ver_pose_Go_stone_set[Go_index][1], zoff=0.0)
    if finger_profile.within(Point([0.05,0.69]).buffer(0.055))==True:
        return False
    else:
        return True

In [30]:
def distane_to_bowl_center(point_coordinate):
    return sqrt((point_coordinate[0]-0.05)**2+(point_coordinate[1]-0.69)**2)

stone_img_count = 0
ver_json_error = False
hori_json_error = False
if 'switching_stone_coordinate_set' in dir():
    while True:
        whether_inherent = raw_input("Whether inherent switching_stone_coordinate_set before? (y or n)")
        if whether_inherent == 'n':
            switching_stone_coordinate_set=[]
            break
        elif whether_inherent == 'y':
            break
else:
    switching_stone_coordinate_set = []
while True:
    while True:
        if_continue = raw_input("continue?")
        if if_continue != '':
            break
    go_to_safe_place()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    hori_Go_stone_number = None
    ver_Go_stone_number = None
    Go_stone_dir_sorted = []
    hori_pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    hori_box_set = []
    ver_pose_Go_stone_set = []
    gripper_direction_set = []
    gripper_aperture_set = []
    is_send_stone_img = 1
    print 'started', stone_img_count
    while True:
        if hori_Go_stone_number != None and hori_Go_stone_number == len(hori_box_set) and ver_Go_stone_number != None and ver_Go_stone_number == len(gripper_aperture_set):
            break
        elif hori_Go_stone_number != None and hori_Go_stone_number == len(hori_box_set) and ver_Go_stone_number != None and ver_json_error==True:
            rospy.sleep(0.5)
            break   
        elif hori_Go_stone_number != None and hori_json_error==True:
            rospy.sleep(0.5)
            break   
        rospy.sleep(0.1)   
    print hori_pose_Go_stone_set
    print yaw_pitch_normal_set
    print ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    priority_list = []
    if ver_pose_Go_stone_set == []:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            priority_list.append(('h', index_hori))
    elif hori_pose_Go_stone_set == []:
        for index_ver in range(len(ver_pose_Go_stone_set)):
            priority_list.append(('v', index_ver))
    else:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            for index_ver in range(len(ver_pose_Go_stone_set)):
                if hori_pose_Go_stone_set[index_hori][2]<=ver_pose_Go_stone_set[index_ver][2]-0.002 and ('v', index_ver) not in priority_list:
                    priority_list.append(('v', index_ver))
                v_num = 0
                for element in priority_list:
                    if element[0]=='v':
                        v_num+=1
                if (v_num == len(ver_pose_Go_stone_set) or hori_pose_Go_stone_set[index_hori][2]>ver_pose_Go_stone_set[index_ver][2]-0.002) and ('h', index_hori) not in priority_list:
                    priority_list.append(('h', index_hori))
        for index_ver in range(len(ver_pose_Go_stone_set)):
            if ('v', index_ver) not in priority_list:
                priority_list.append(('v', index_ver))
    print priority_list
    for index in priority_list:
        '''
        if index[0]=='v':
            if ver_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue
            if ver_pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if ver_pose_Go_stone_set[index[1]][2]>0.075:
                continue
            if direct_pick_collision_check(index[1])==False:
                direct_pick_execute(index[1])
                break
        '''
        if index[0]=='h':
            if hori_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue  
            if hori_pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if hori_pose_Go_stone_set[index[1]][2]>0.075:
                continue
            if distane_to_bowl_center(hori_pose_Go_stone_set[index[1]])<0.05:
                scoop4(index[1])
                break
            '''
            else:
                if scoop_collision_check(index[1])==False:
                    scoop_execute(index[1]) 
                    break
            '''

Whether inherent switching_stone_coordinate_set before? (y or n)n
continue?1
started 0
dsag
0
[[0.023567786778194467, 0.682421107545821, -0.29897991247045147], [0.0660945121987683, 0.7080379855907365, 0.03821861297514989], [0.06620392468146412, 0.7427309922034518, 0.029615964681256268], [0.08394517617033899, 0.664738420476875, 0.03240211104261076]]
[[-1.4550371090740861, 0.446105548943404, (-0.10562801314344909, 0.0165836332164313, -0.9750882584058667)], [-2.9094888155146377, 0.446105548943404, (-0.015406254249173398, 0.047805803818730785, -0.9152561185467837)], [1.2357722157140798, 0.4808872801953352, (-0.004576729331605193, -0.001476654405563604, -0.928909006383036)], [1.786881299091683, 0.4101273405414903, (0.008382273891184629, 0.012811384129282217, -0.9723381815631214)]]
[] [] []
[('h', 0), ('h', 1), ('h', 2), ('h', 3)]
Go_index 1
worldPstone [0.06609451 0.70803799 0.03821861 1.        ]
tcpTstone [[0.16184409]
 [0.02213439]
 [0.12664537]
 [1.        ]]
<Transform:
<Orientation: 


KeyboardInterrupt: 

In [22]:
((0.075-0.07522677506936579)+(0.05-0.050549765559007184)+(0.025-0.024505401570517785)+(0.025-0.02616018528975756))/4.0

-0.00036053187216207854

In [23]:
((0.675-0.6743803927431379)+(0.675-0.6735048336785908)+(0.7-0.6976843453451326)+(0.65-0.6473276838358174))/4.0

0.001775686099330348

In [30]:
Robotiq.goto(robotiq_client, pos=0.021, speed=0.000001, force=1, block=True)
#servo_pub.publish(180)
#softtip()

In [41]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.000001, force=1, block=True)

In [18]:
tcpTnormal

NameError: name 'tcpTnormal' is not defined

In [9]:
go_to_safe_place()
hardtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
rob.translate_tool((0.0065,0,0),acc=0.1, vel=0.2)

<Transform:
<Orientation: 
array([[-0.01308933,  0.99989511,  0.00619996],
       [ 0.99991303,  0.01307907,  0.00169334],
       [ 0.00161207,  0.00622159, -0.99997935]])>
<Vector: (0.04282, 0.80750, 0.45825)>
>

In [75]:
rob.set_tcp((-0.02, 0, 0.342, 0, 0, 0))
rospy.sleep(0.5)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [37]:
        hardtip()
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001

        rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)     


<Transform:
<Orientation: 
array([[-0.01329421,  0.99989335,  0.00604565],
       [ 0.99990707,  0.01327565,  0.00310065],
       [ 0.00302006,  0.00608631, -0.99997692]])>
<Vector: (0.07290, 0.66112, 0.45890)>
>

In [20]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [74]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=4*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = -0.02 # need change
tcp2fingertip.pos.z = 0.342 # need change
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.004
world2tcp.pos.y+=0.001
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
#rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)

<Transform:
<Orientation: 
array([[-0.01299897,  0.99989554,  0.00632014],
       [ 0.99991464,  0.01299036,  0.00140147],
       [ 0.00131922,  0.00633782, -0.99997905]])>
<Vector: (0.04338, 0.86854, 0.45810)>
>

In [5]:
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    print tcp2cam

[[ 0.02153275 -0.9995875  -0.01900464  0.1234057 ]
 [ 0.99884698  0.02232484 -0.04250071 -0.0074811 ]
 [ 0.04290745 -0.01806757  0.99891567  0.126173  ]
 [ 0.          0.          0.          1.        ]]


In [3]:
softtip()

KeyboardInterrupt: 

In [42]:
rob.set_tcp((0.006, 0, 0.3734, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.5, vel=0.8, wait=True)
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.5, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [29]:
go_to_safe_place()

In [31]:
softtip()
hardtip()

In [153]:
    servo_pub.publish(180)
    rospy.sleep(1)


In [116]:
rob.set_tcp((0, 0, 0.35947, 0, 0, 0))
rospy.sleep(1)
print rob.get_pose()
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(1)

<Transform:
<Orientation: 
array([[-0.01308334,  0.99989484,  0.00625631],
       [ 0.99991303,  0.01307268,  0.00174152],
       [ 0.00165955,  0.00627855, -0.99997891]])>
<Vector: (0.04541, 0.54392, 0.09884)>
>


In [24]:
import serial
import signal
import time
serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
time0 = time.time()
def myHandler(signum, frame):
    pass
signal.signal(signal.SIGALRM, myHandler)
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)
print time.time()-time0
print '2'

1.00144481659
2


In [21]:

softtip()

In [24]:
hori_pose_Go_stone_set[index[1]][0]>point0[0]-0.015 and hori_pose_Go_stone_set[index[1]][0]<point0[0]+0.015 and hori_pose_Go_stone_set[index[1]][1]>point0[1]-0.015 and hori_pose_Go_stone_set[index[1]][1]<point0[1]+0.015 and hori_pose_Go_stone_set[index[1]][2]<point0[2]

True

In [28]:
#go_to_safe_place()
rob.set_tcp((0, 0, 0.36142, 0, 0, 0))
rospy.sleep(0.5)

move = m3d.Transform((0.8-rob.get_pose().pos.y+0.001, 0.05-rob.get_pose().pos.x-0.004, 0, 0, 0, 0))    
rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [39]:
rob.set_tcp((0, 0, 0.36142, 0, 0, 0))
rospy.sleep(0.5)
rob.get_pose()

<Transform:
<Orientation: 
array([[-0.01297587,  0.99989633,  0.00624184],
       [ 0.99991459,  0.01296587,  0.00163983],
       [ 0.00155873,  0.00626258, -0.99997917]])>
<Vector: (0.04543, 0.54389, 0.09685)>
>

In [23]:
go_to_end_place()

In [17]:
      Robotiq.goto(robotiq_client, pos=0.8, speed=0.5, force=10, block=True)


In [20]:
softtip()

In [30]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1000')
time.sleep(1)
signal.setitimer(signal.ITIMER_REAL, 0)
for k0 in range(0, 81, 4):
    signal.setitimer(signal.ITIMER_REAL, 0.001)
    ser.write('1'+str(k0))
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.2)
print 4

4


In [89]:
time0=time.time()
signal.setitimer(signal.ITIMER_REAL, 0)
for k0 in range(0, -31, -2):
    signal.setitimer(signal.ITIMER_REAL, 0.001)
    ser.write('1'+str(100+k0))
    print '1'+str(100+k0)
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)
print 4

1100
198
196
194
192
190
188
186
184
182
180
178
176
174
172
170
4


In [30]:
go_to_safe_place()
rob.translate((0,0,-0.035), acc=0.1, vel=0.1, wait=True)

<Transform:
<Orientation: 
array([[-0.01306666,  0.99989477,  0.00630122],
       [ 0.99991333,  0.01305622,  0.00169527],
       [ 0.00161282,  0.00632282, -0.99997871]])>
<Vector: (0.04322, 0.54326, 0.42337)>
>

In [17]:
switching_stone_coordinate_set

[[0.04748682930698754, 0.6772187228828948, 0.03650951687510667],
 [0.04285301004949358, 0.6773116203834977, 0.04474243949710238],
 [0.020558080749336436, 0.7007856115591724, 0.05011494897992741],
 [0.04654879802776229, 0.7035219271723925, 0.06111810419601599],
 [0.03623374456712877, 0.6653812126120116, 0.05419025420948426],
 [0.06196829769648066, 0.687646955878628, 0.053823396522124034],
 [0.05775511489062693, 0.6660403423087387, 0.06140280538363507],
 [0.06572986499869145, 0.7087816075687934, 0.04626566310967978],
 [0.0769745887897275, 0.6758826107559193, 0.046508792057820714],
 [0.022329528772110585, 0.6842764932194296, 0.04333823667183745],
 [0.04992898805327124, 0.7293906219163122, 0.03750523331270783],
 [0.023780934590662446, 0.7260079873804601, 0.03652088731031233],
 [0.005551637499277247, 0.6835217107176383, 0.03491895253859206],
 [0.01950094174002195, 0.660207221543091, 0.036879290367629936],
 [0.03986341513255099, 0.6496814636328898, 0.036299742084113296],
 [0.0739991948029581

In [32]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)


In [48]:
time0 = time.time()
rob.translate((0, 0, -0.015), acc=0.01, vel=0.05, wait=True)
print time.time()-time0

2.68596196175


In [31]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [30]:
Robotiq.goto(robotiq_client, pos=0.047, speed=0.5, force=10, block=True)

In [17]:
softtip()

In [7]:
go_to_safe_place()

In [43]:
softtip()

In [46]:
(0.047-1.12/48.0)*48/0.14

8.114285714285712

In [32]:
go_to_home()